### Import libraries

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import csv
import re
from time import sleep
from datetime import date, timedelta

# Ignore certification validation
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
quiniela_argument = 'nacional'
period_argument = 'primera'
results_date = date(2014, 11, 2) # Oldest date with data in the website 
# end_date = date(2014, 11, 3)
end_date = date.today()
date_delta = timedelta(days=1)

In [3]:
# Set HTML parser
# Create and set the structure of the file to be exported with scraped data
file = open('{}-{}.csv'.format(quiniela_argument, period_argument), 'w', newline='')
writer = csv.writer(file)
quiniela_file_headers = ['date', 'quiniela', 'period', 'position', 'result']
writer.writerow(quiniela_file_headers)

while results_date <= end_date:
    results_date += date_delta 

    base_url = 'https://www.nacionalloteria.com/argentina/quiniela-{}.php?del-dia={}&periodo={}'.format(quiniela_argument, results_date, period_argument)
    html = urllib.request.urlopen(base_url, timeout=10, context=ctx).read()
    soup = BeautifulSoup(html, 'lxml')

    quiniela_file_rows = []

    rows = soup.find_all('td', class_='celNor tacenter')
    
    i = 0
    while i < 42:
        quiniela_file_rows = [results_date.strftime('%Y-%m-%d'), quiniela_argument, period_argument, rows[i].text, rows[i+1].text]
        i += 2
        writer.writerow(quiniela_file_rows)

file.close() # Save and close file

In [ ]:
with open('lottery_numbers_page_template.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))